# Bike Sharing (MLFlow - KServe)

This notebook offers a detailed walkthrough of a comprehensive data science workflow. It covers data preprocessing, model training and evaluation, hyperparameter tuning, experiment tracking with MLFlow, and model deployment using Seldon and KServe. It focuses on the well-known bike sharing dataset from the UCI Machine Learning Repository.

![bike-sharing](images/bike-sharing.jpg)
(Photo by <a href="https://unsplash.com/@zaccastravels?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText">ZACHARY STAINES</a> on <a href="https://unsplash.com/photos/KEhNcoCldbk?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText">Unsplash</a>)

The dataset captures the hourly and daily count of rental bikes from 2011 to 2012 in the Capital Bikeshare system. It also includes corresponding weather and seasonal data. The dataset aims to encourage research on bike sharing systems, which are increasingly important for traffic management, environmental sustainability, and public health.

The task for this dataset is regression and contains 17,389 instances. The main goal is to build a predictive model that can accurately forecast bike rental demand. The primary target variable for prediction is the `cnt` attribute, which represents the total count of rental bikes, including both casual and registered users.

By leveraging additional features in the dataset, such as time and weather information, you will train a model to predict this count with a high level of accuracy.

**References:**
- https://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset
- https://docs.databricks.com/_static/notebooks/gbt-regression.html
- https://www.kaggle.com/pratsiuk/mlflow-experiment-automation-top-9
- https://mlflow.org/docs/latest/tracking.html

## Setting Up the Environment

The following code cells focus on importing the necessary dependencies. Also, you should set up a local directory to save the training artifacts generated during your experiment.

In [ ]:
import os
import json
import base64
import datetime
import subprocess
import itertools
import warnings

from urllib.parse import urlparse
from functools import partial

import git
import boto3
import kserve
import sklearn
import ipywidgets as widgets
import s3transfer
import requests
import graphviz
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import mlflow
import mlflow.sklearn

from mlflow.models.signature import infer_signature
from mlflow import log_metric, log_param, log_artifact
from sklearn import tree
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.inspection import permutation_importance
from pydotplus import graph_from_dot_data
from IPython.display import Image, display
from kubernetes import client, config
from kubernetes.client import V1ObjectMeta, V1Secret, V1ServiceAccount, V1ObjectReference, V1Container, V1EnvVar
from kserve import (V1beta1InferenceService, V1beta1InferenceServiceSpec, V1beta1SKLearnSpec,
                    V1beta1PredictorSpec, V1beta1ModelSpec, V1beta1ModelFormat)


plt.style.use("fivethirtyeight")
pd.plotting.register_matplotlib_converters()

warnings.filterwarnings('ignore')

In [ ]:
if os.path.exists("model_artifacts"):
    os.system("rm -rf model_artifacts")
os.mkdir("model_artifacts")

## Set the MLflow experiment

First, set up an experiment using MLFlow to track various runs during the model training process. MLFlow is an open-source platform that manages the end-to-end machine learning lifecycle.

In [ ]:
# Set up an MLflow experiment to track model training runs
experiment_name = 'bike-sharing-exp'
mlflow.set_experiment(experiment_name)

## Load the Dataset

With the preliminary setup complete, you can now move on to loading the dataset. The data comes in CSV format, easily loadable using the Pandas library in Python. To get a quick look at the dataset, you can display the first five rows using the `head()` method of the DataFrame. This initial exploration gives you a snapshot of the data you'll work with.

In [ ]:
# Load input data into pandas dataframe
bike_sharing = pd.read_csv(os.path.join("data", "bike-sharing.csv"))
bike_sharing.head()

## Data preprocessing

In this phase, you'll prepare the data for the subsequent stages of your analysis. This involves cleaning, transforming, and structuring the data to make sure it's in the optimal format for your machine learning model.

In [ ]:
# Remove unused columns
bike_sharing.drop(columns=["instant", "dteday", "registered", "casual"], inplace=True)

# Use better names
bike_sharing.rename(
    columns={
        "yr": "year",
        "mnth": "month",
        "hr": "hour_of_day",
        "holiday": "is_holiday",
        "workingday": "is_workingday",
        "weathersit": "weather_situation",
        "temp": "temperature",
        "atemp": "feels_like_temperature",
        "hum": "humidity",
        "cnt": "rented_bikes",
    }, inplace=True)

# Convert every data point to `float64`
cols = bike_sharing.select_dtypes(exclude=['float64']).columns
for i in ['season', 'year', 'month', 'hour_of_day', 'is_holiday',
          'weekday', 'is_workingday', 'weather_situation', 'rented_bikes']:
    bike_sharing[i] = bike_sharing[i].astype('float64')

## Data Visualization

In this section, you'll use various visualization techniques to gain a deeper understanding of your data. Creating graphical representations will help you identify patterns, trends, and correlations that might not be obvious in the raw data. This step is crucial for guiding your subsequent analysis and model-building efforts.

In [ ]:
hour_of_day_agg = bike_sharing.groupby(["hour_of_day"])["rented_bikes"].sum()

hour_of_day_agg.plot(
    kind="line", 
    title="Total rented bikes by hour of day",
    xticks=hour_of_day_agg.index,
    figsize=(10, 5),
)

plt.show()

## Prepare training and test data sets

In this section, you'll partition your data into training and test datasets. This step is crucial in the machine learning workflow. It allows you to train your model on a subset of the data, known as the training set, and then evaluate its performance on unseen data, or the test set. This approach helps ensure that your model generalizes well to new data and avoids simply memorizing the training data.

In [ ]:
# Split the dataset randomly into 70% for training and 30% for testing.
X = bike_sharing.drop("rented_bikes", axis=1)
y = bike_sharing.rented_bikes
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=42)

print(f"Training samples: {X_train.size}")
print(f"Test samples: {X_test.size}")

## Establishing Evaluation Metrics

Before moving on to the training stage, you'll define the evaluation metrics to assess your model's performance. These metrics offer quantitative measures of the model's accuracy, helping you understand its effectiveness and areas for improvement. This step is crucial for making sure your model meets the desired performance standards.

### Root Mean Square Error (RMSE)

One evaluation metric you'll use is the Root Mean Square Error (RMSE). This metric quantifies the differences between the values your model predicts and the actual values. By taking the square root of the average of these squared differences, RMSE helps you gauge the magnitude of the prediction errors. Lower RMSE values signify a better fit of the model to the data.

References: 
- https://medium.com/@xaviergeerinck/artificial-intelligence-how-to-measure-performance-accuracy-precision-recall-f1-roc-rmse-611d10e4caac
- https://www.kaggle.com/residentmario/model-fit-metrics#Root-mean-squared-error-(RMSE)

In [ ]:
def rmse(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

def rmse_score(y, y_pred):
    score = rmse(y, y_pred)
    message = "RMSE score: {:.4f}".format(score)
    return score, message

### Cross-Validation RMSLE score

Another evaluation metric you'll use is the Root Mean Squared Logarithmic Error (RMSLE) score, calculated through cross-validation. Cross-validation is a robust technique that averages measures of prediction accuracy to provide a more accurate estimate of model performance.

The RMSLE score is particularly useful in your case because it penalizes underestimates more than overestimates. This makes it an essential metric for your bike sharing demand prediction model, helping you avoid situations where the available number of bikes doesn't meet the demand.

References: 
- https://en.wikipedia.org/wiki/Cross-validation_(statistics)
- https://www.kaggle.com/carlolepelaars/understanding-the-metric-rmsle


In [ ]:
def rmsle_clip(estimator, x, y):
    """Clip negative prediction numbers before calculating RMSLE."""
    y_pred = estimator.predict(x)
    y_pred_clipped = np.clip(y_pred, a_min=0, a_max=None)
    return sklearn.metrics.mean_squared_log_error(y, y_pred_clipped, squared=False)

def rmsle_cv(model, X_train, y_train):
    kf = KFold(n_splits=4, shuffle=True, random_state=42).get_n_splits(X_train.values)
    # Evaluate RMSLE score by cross-validation
    rmsle = cross_val_score(model, X_train.values, y_train, scoring=rmsle_clip, cv=kf, error_score="raise")
    return rmsle

def rmsle_cv_score(model, X_train, y_train):
    score = rmsle_cv(model, X_train, y_train)
    message = "Cross-Validation RMSLE score: {:.4f} (std = {:.4f})".format(score.mean(), score.std())
    return score, message

## Feature Importance

In this section, you'll analyze the importance of each feature in your dataset. Feature importance involves techniques that assign a score to input features based on their usefulness in predicting a target variable.

Knowing which features most strongly influence the target variable can give you valuable insights into your dataset and the underlying model. This understanding can help you interpret the model's predictions and guide future data collection and feature engineering efforts.

References:
- https://medium.com/bigdatarepublic/feature-importance-whats-in-a-name-79532e59eea3

In [ ]:
def model_feature_importance(model):
    feature_importance = pd.DataFrame(
        model.feature_importances_,
        index=X_train.columns,
        columns=["Importance"])

    # sort by importance
    feature_importance.sort_values(by="Importance", ascending=False, inplace=True)

    # plot
    plt.figure(figsize=(10, 4))
    sns.barplot(
        data=feature_importance.reset_index(),
        y="index",
        x="Importance",
    ).set_title("Feature Importance")

    # save image
    plt.savefig("model_artifacts/feature_importance.png", bbox_inches='tight')

## Permutation Importance

Permutation Importance is a technique you'll use to measure feature importance. This method randomly shuffles a single feature in the validation data and measures the resulting decrease in the model's performance. Features that lead to the most significant drop in performance are considered the most important.

This approach offers a straightforward way to understand the impact of each feature on the model's predictions. It can help you identify which features are driving the model's decisions and where you might concentrate your efforts for further data analysis or feature engineering.

References:
- https://www.kaggle.com/dansbecker/permutation-importance

In [ ]:
def model_permutation_importance(model):
    p_importance = permutation_importance(model, X_test, y_test, random_state=42, n_jobs=-1)

    # sort by importance
    sorted_idx = p_importance.importances_mean.argsort()[::-1]
    p_importance = pd.DataFrame(
        data=p_importance.importances[sorted_idx].T,
        columns=X_train.columns[sorted_idx]
    )

    # plot
    plt.figure(figsize=(10, 4))
    sns.barplot(
        data=p_importance,
        orient="h"
    ).set_title("Permutation Importance")

    # save image
    plt.savefig("model_artifacts/permutation_importance.png", bbox_inches="tight")

## Decision Tree Visualization

In this section, you'll visualize the decision tree model. A decision tree resembles a flowchart, where each internal node represents a feature, each branch signifies a decision rule, and each leaf node indicates an outcome.

Visualizing the decision tree helps you clearly understand how the model makes predictions. You can see the decision paths and rules the model employs, making it one of the most interpretable machine learning models. This can be especially helpful when you need to explain the model's decisions to stakeholders.

References:
- https://towardsdatascience.com/visualizing-decision-trees-with-python-scikit-learn-graphviz-matplotlib-1c50b4aa68dc 

In [ ]:
def model_tree_visualization(model):
    # generate visualization
    tree_dot_data = tree.export_graphviz(
        # Get the first tree
        # TODO: Visualize every tree
        decision_tree=model.estimators_[0, 0],
        label="all",
        feature_names=X_train.columns,
        filled=True,
        rounded=True,
        proportion=True,
        impurity=False,
        precision=1,
    )

    # save image
    graph_from_dot_data(tree_dot_data).write_png("model_artifacts/Decision_Tree_Visualization.png")

    # show tree
    return graphviz.Source(tree_dot_data)

## MLflow Tracking

In this phase, you'll use MLflow Tracking, a component of MLflow designed to log and track experiment data. This data includes parameters, metrics, and artifacts generated during the model training process.

MLflow Tracking offers a centralized repository for metadata related to your experiments. This makes it easier to compare different runs, reproduce results, and share findings with your team. This step is vital for maintaining an organized and efficient machine learning workflow.

First, let's set up the logger.

References:
- https://www.mlflow.org/docs/latest/cli.html#mlflow-ui

In [ ]:
# Track params and metrics
def log_mlflow_run(model, signature):
    # Auto-logging for scikit-learn estimators
    # mlflow.sklearn.autolog()

    # log estimator_name name
    name = model.__class__.__name__
    mlflow.set_tag("estimator_name", name)

    # log input features
    mlflow.set_tag("features", str(X_train.columns.values.tolist()))

    # Log tracked parameters only
    mlflow.log_params({key: model.get_params()[key] for key in parameters})

    mlflow.log_metrics({
        'RMSLE_CV': score_cv.mean(),
        'RMSE': score})

    # log training loss
    for s in model.train_score_:
        mlflow.log_metric("Train Loss", s)

    # Save model to artifacts
    mlflow.sklearn.log_model(model, "model")#, signature=signature)

    # log charts
    mlflow.log_artifacts("model_artifacts")

    # misc
    # Log all model parameters
    # mlflow.log_params(model.get_params())
    mlflow.log_param("Training size", X_test.size) 
    mlflow.log_param("Test size", y_test.size)

## Model Training and Hyperparameter Tuning

In this section, you'll focus on training your model and tuning its hyperparameters. For this specific use case, you'll use the following approach:

- Approach: You'll employ a Supervised Learning method, specifically a Decision Tree model. Decision Trees are intuitive and easy-to-understand models that make decisions based on a set of rules derived from the features.
- Tree Type: Since your task involves predicting a continuous target variable—the total count of rental bikes—you'll use a Regression Tree. Regression Trees predict outcomes by learning simple decision rules based on the features.
- Technique/Ensemble Method: To enhance the performance of your Decision Tree model, you'll use an ensemble method called Gradient Boosting. This technique combines several weak learners—in this case, Decision Trees—to create a robust predictive model. It trains models in a gradual, additive, and sequential manner, with each new model correcting the errors of the previous ones.

By carefully tuning the hyperparameters of your Gradient Boosting model, you can optimize its performance and ensure it generalizes well to new data.

References:
- GBRT (Gradient Boosted Regression Tree): https://orbi.uliege.be/bitstream/2268/163521/1/slides.pdf
- Choosing a model: https://scikit-learn.org/stable/tutorial/machine_learning_map
- Machine Learning Models Explained
: https://docs.paperspace.com/machine-learning/wiki/machine-learning-models-explained
- Gradient Boosted Regression Trees: https://orbi.uliege.be/bitstream/2268/163521/1/slides.pdf


In [ ]:
# GBRT (Gradient Boosted Regression Tree) scikit-learn implementation 
model_class = GradientBoostingRegressor

Set the training's process hyperparameters.

In [ ]:
parameters = {
    "learning_rate": [0.1, 0.05, 0.01],
    "max_depth": [4, 5, 6],
    # "verbose": True,
}

To optimize your model's performance, you'll tune its hyperparameters using Grid Search.

Grid Search is a traditional method for hyperparameter tuning that defines a grid of hyperparameters and evaluates the model's performance at each grid point. You can think of this as an exhaustive search through a manually specified subset of the hyperparameter space for the chosen algorithm.

By using Grid Search, you can systematically explore multiple combinations of hyperparameters to find the optimal values that improve your model's performance. This process can significantly enhance the predictive accuracy of your model.

References:
- More advanced tuning techniques: https://research.fb.com/efficient-tuning-of-online-systems-using-bayesian-optimization/

In [ ]:
# generate parameters combinations
params_keys = parameters.keys()
params_values = [
    parameters[key] if isinstance(parameters[key], list) else [parameters[key]]
    for key in params_keys]

runs_parameters = [
    dict(zip(params_keys, combination)) for combination in itertools.product(*params_values)]

## Model Training

Now that you've prepared your data and set up your model, the next step is to train the model. During this process, the model will learn from the features in your training data to predict the target variable.

Training the model involves adjusting it to minimize the difference between the predicted and actual values, guided by a specific learning algorithm. In your case, you're using a Gradient Boosting model, which will learn to correct its errors in a gradual, additive, and sequential manner.

This step is crucial in your machine learning workflow, as the quality of your model's predictions largely depends on the effectiveness of the training process.

In [ ]:
progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=len(runs_parameters),
    # description='Training progress',
    bar_style='info',
    orientation='horizontal'
)

progress_text = widgets.Output()

display(progress_bar, progress_text)

# training loop
for i, run_parameters in enumerate(runs_parameters):
    progress_bar.description = f"Run {i+1}/{len(runs_parameters)}"
    # mlflow: stop active runs if any
    if mlflow.active_run():
        mlflow.end_run()
    # mlflow:track run
    mlflow.start_run(run_name=f"Run {i}")

    # create model instance
    model = model_class(**run_parameters)

    # train
    model.fit(X_train, y_train)

    # get evaluations scores
    ypred = model.predict(X_test)
    score, message = rmse_score(y_test, model.predict(X_test))
    score_cv, message_cv = rmsle_cv_score(model, X_train, y_train)
    
    # generate charts
    model_feature_importance(model)
    plt.close()
    model_permutation_importance(model)
    plt.close()
    # model_tree_visualization(model)

    # get model signature
    signature = infer_signature(model_input=X_train, model_output=model.predict(X_train))

    # mlflow: log metrics
    log_mlflow_run(model, signature)

    # mlflow: end tracking
    mlflow.end_run()
    
    progress_bar.value = i + 1
    
    # Update progress text
    with progress_text:
        print(f"Learning rate: {run_parameters['learning_rate']}"
              f"\tMax depth: {run_parameters['max_depth']}")
        print(message)
        print(message_cv)
        print()

## Best Model Results

After training several models and tuning their hyperparameters, you'll identify the model that performs best according to your chosen evaluation metrics.

In [ ]:
best_run_df = mlflow.search_runs(order_by=['metrics.RMSLE_CV ASC'], max_results=1)
if len(best_run_df.index) == 0:
    raise Exception(f"Found no runs for experiment '{experiment_name}'")

best_run = mlflow.get_run(best_run_df.at[0, 'run_id'])
best_model_uri = f"{best_run.info.artifact_uri}/model"
best_model = mlflow.sklearn.load_model(best_model_uri)

In [ ]:
# Print best run info
print("Best run info:")
print(f"Run id: {best_run.info.run_id}")
print(f"Run parameters: {best_run.data.params}")
print(f"Run score: RMSLE_CV = {best_run.data.metrics['RMSLE_CV']:.4f}")
print(f"Run model URI: {best_model_uri}")

In [ ]:
model_feature_importance(best_model)

In [ ]:
model_permutation_importance(best_model)

In [ ]:
model_tree_visualization(best_model)

## Model Testing

Once you've identified your best model, the next step is to test its predictive performance on unseen data using the test dataset, which you set aside specifically for this purpose.

Testing the model's predictions allows you to assess how well the model generalizes to new data. This step is crucial in the machine learning process because it provides a realistic estimate of the model's performance in real-world settings.

You'll compare the model's predictions with the actual values in the test dataset and calculate your chosen evaluation metrics. These results will give you a clear indication of the model's predictive accuracy.

In [ ]:
test_predictions = X_test.copy()
# real output (rented_bikes) from test dataset
test_predictions["rented_bikes"] = y_test

# add "predicted_rented_bikes" from test dataset
test_predictions["predicted_rented_bikes"] = best_model.predict(X_test).astype(int)

# show results
test_predictions.head()

In [ ]:
# plot truth vs prediction values
test_predictions.plot(
    kind="scatter",
    x="rented_bikes",
    y="predicted_rented_bikes",
    title="Rented bikes vs predicted rented bikes",
    figsize=(10, 10)
)

plt.show()

## Model Deployment

In this section of the notebook, you'll focus on deploying the trained model to bridge the gap between insightful data analysis and real-world impact. To do this, you'll use KServe, an open-source platform that simplifies the deployment and management of machine learning models at scale. KServe offers a robust and scalable infrastructure for serving predictions from trained models in production environments. For KServe's backend, you'll use Seldon.

In [ ]:
isvc = """
apiVersion: v1
kind: ServiceAccount
metadata:
  name: kserve-minio-secret
secrets:
- name: {0}-objectstore-secret

---
apiVersion: serving.kserve.io/v1beta1
kind: InferenceService
metadata:
  name: bike-sharing-exp
spec:
  predictor:
    serviceAccountName: kserve-minio-secret
    sklearn:
      protocolVersion: v2
      storageUri: {1}
""".format(os.environ['USER'], best_model_uri)

with open(os.path.join("manifests", "isvc.yaml"), "w") as f:
    f.write(isvc)

In [ ]:
subprocess.run(["kubectl", "apply", "-f", "manifests/isvc.yaml"])